In [ ]:
import pandas as pd
import numpy as np
from time import sleep
from numpy import inf
from datetime import date,datetime, timedelta

from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adset import AdSet
from facebook_business.api import FacebookAdsApi

# FBIG API 활용, 광고 관리자 데이터 가져오기

In [ ]:
#### 페이스북 광고 관리자 데이터 가져오기

my_app_id = '****'
my_app_secret = '****'
my_access_token = '****'
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

####  FBIG 광고 계정 2개 운영
AdAccount_list = [
    'act_****'
    ,'act_****'
]

df_fb_budget_dic = {}

for adaccount in AdAccount_list:
    adset = []
    date_ = ''
    temp_fb_data = []
    params = {}

    for i in range(14):
        date_ = (date.today() - timedelta(days=14-i)).strftime("%Y-%m-%d")
        params = {
            'level':'adset',
            'limit':100,
            'time_range':{
                'since':date_,
                'until':date_
            }
        }
        temp_fb_data = []
        temp_fb_data = AdAccount(adaccount).get_insights(
            fields=[
                'account_id',
                'adset_id',
                'adset_name',
                'campaign_id',
                'spend'
            ],
            params=params
        )
        sleep(1)
        for c in range(len(temp_fb_data)):
            adset.append(list(temp_fb_data[c].values()))
    df_fb_budget_dic[adaccount] = pd.DataFrame(adset,columns=(
        'Adaccount ID',
        'Adset ID',
        '광고 세트 이름',
        '캠페인 ID' ,
        '지출 금액 (KRW)',
        '보고 시작',
        'Date'
    ))

df_fb_budget = pd.concat([
    df_fb_budget_dic['act_****'],
    df_fb_budget_dic['act_****']
])

df_fb_budget['지출 금액 (KRW)'] = df_fb_budget['지출 금액 (KRW)'].astype(int)

fb_adset_daily_budget = []
fb_adset_list = []

for Account,Adset_id in df_fb_budget[['Adaccount ID','Adset ID']].drop_duplicates(['Adset ID']).to_numpy():
    fb_adset_list.append(Adset_id)
    my_account = AdAccount('act_'+Account)
    adset = AdSet(fbid=Adset_id)
    fields = [
        AdSet.Field.daily_budget
    ]
    params = {
        'date_format':'U'
    }
    adset.api_get(fields=fields, params=params)
#### 리타겟팅 광고 관리자 데이터 분리
    if Account =='****':
        fb_adset_daily_budget.append(adset[AdSet.Field.daily_budget])
    else:
        fb_adset_daily_budget.append(0)
    sleep(0.5)
    
df_fb_daily_budget = pd.DataFrame(data={'Adset ID':fb_adset_list,'daily_budget':fb_adset_daily_budget})

df_fb_budget = pd.merge(
    df_fb_budget,
    df_fb_daily_budget[['Adset ID','daily_budget']],
    'left',
    on='Adset ID',
    suffixes=('','_y')
)

# 매체별, Score, Score_assist 성과 분석 후, 성과 분석용 트렌드 지표 생성

In [ ]:
#### FBIG 광고 성과 평가 지표
#### 광고 그룹별 (광고 그룹단위로 예산 조절)
#### 3가지 이벤트 (설치, 견적요청 중간 단계, 견적요청 완료) 전환 단가를
#### 최근 3일, 최근 7일 기준
#### {최근 3일, 7일} 의 7일 간 트렌드 참고하여 Score

for i in range(7):
    yesterday = (dt.datetime.today() - pd.Timedelta(str(1+i) + ' days')).strftime("%Y-%m-%d")
    _3days_ago = (dt.datetime.today() - pd.Timedelta(str(3+i) +' days')).strftime("%Y-%m-%d")
    _7days_ago = (dt.datetime.today() - pd.Timedelta(str(7+i) + ' days')).strftime("%Y-%m-%d")

    df_merge_budget_3days = df_merge_budget[(df_merge_budget['일']>=_3days_ago)&(df_merge_budget['일']<=yesterday)].pivot_table(index=['publisher.name_facebook_filtering','A_columns','B_columns'],values=['비용','노출','클릭'],aggfunc='sum').reset_index()
    df_merge_budget_7days = df_merge_budget[(df_merge_budget['일']>=_7days_ago)&(df_merge_budget['일']<=yesterday)].pivot_table(index=['publisher.name_facebook_filtering','A_columns','B_columns'],values=['비용','노출','클릭'],aggfunc='sum').reset_index()
#### A_columns : 캠페인
#### B_columns : 광고 그룹
    df_result_install_in3 = df_result[(df_result['****date']>=_3days_ago)&(df_result['****date']<=yesterday)].pivot_table(index=['publisher.name_facebook_filtering','A_columns','B_columns'],values='created',aggfunc='count').reset_index()
    df_result_install_in7 = df_result[(df_result['****date']>=_7days_ago)&(df_result['****date']<=yesterday)].pivot_table(index=['publisher.name_facebook_filtering','A_columns','B_columns'],values='created',aggfunc='count').reset_index()
    df_result_request_3step_in3 = df_result[(df_result['****date']>=_3days_ago)&(df_result['****date']<=yesterday)&(df_result['7일이내_견적요청_3단계_여부']==True)].pivot_table(index=['publisher.name_facebook_filtering','A_columns','B_columns'],values='created',aggfunc='count').reset_index()
    df_result_request_3step_in7 = df_result[(df_result['****date']>=_7days_ago)&(df_result['****date']<=yesterday)&(df_result['7일이내_견적요청_3단계_여부']==True)].pivot_table(index=['publisher.name_facebook_filtering','A_columns','B_columns'],values='created',aggfunc='count').reset_index()
    df_result_request_in3 = df_result[(df_result['****date']>=_3days_ago)&(df_result['****date']<=yesterday)&(df_result['7일이내_견적요청_여부']==True)].pivot_table(index=['publisher.name_facebook_filtering','A_columns','B_columns'],values='created',aggfunc='count').reset_index()
    df_result_request_in7 = df_result[(df_result['****date']>=_7days_ago)&(df_result['****date']<=yesterday)&(df_result['7일이내_견적요청_여부']==True)].pivot_table(index=['publisher.name_facebook_filtering','A_columns','B_columns'],values='created',aggfunc='count').reset_index()
    
    df_merge_result = pd.merge(df_merge_budget_7days,df_merge_budget_3days,'left',on=['publisher.name_facebook_filtering','A_columns','B_columns'],suffixes=('_최근7일','_최근3일'))
    df_merge_result = pd.merge(df_merge_result,df_result_install_in3,'left',on=['publisher.name_facebook_filtering','A_columns','B_columns'],suffixes=('',''))
    df_merge_result = pd.merge(df_merge_result,df_result_install_in7,'left',on=['publisher.name_facebook_filtering','A_columns','B_columns'],suffixes=('','_최근7일_인스톨'))
    df_merge_result = pd.merge(df_merge_result,df_result_request_3step_in3,'left',on=['publisher.name_facebook_filtering','A_columns','B_columns'],suffixes=('','_최근3일_3단계'))
    df_merge_result = pd.merge(df_merge_result,df_result_request_3step_in7,'left',on=['publisher.name_facebook_filtering','A_columns','B_columns'],suffixes=('','_최근7일_3단계'))
    df_merge_result = pd.merge(df_merge_result,df_result_request_in3,'left',on=['publisher.name_facebook_filtering','A_columns','B_columns'],suffixes=('','_최근3일_견적요청'))
    df_merge_result = pd.merge(df_merge_result,df_result_request_in7,'left',on=['publisher.name_facebook_filtering','A_columns','B_columns'],suffixes=('_최근3일_인스톨','_최근7일_견적요청'))
    df_merge_result = df_merge_result.fillna(0)
    df_merge_result['최근3일_CPI'] = (df_merge_result['비용_최근3일']/df_merge_result['created_최근3일_인스톨']).replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
    df_merge_result['최근7일_CPI'] = (df_merge_result['비용_최근7일']/df_merge_result['created_최근7일_인스톨']).replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
    df_merge_result['최근3일_3단계_CPA'] = (df_merge_result['비용_최근3일']/df_merge_result['created_최근3일_3단계']).replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
    df_merge_result['최근7일_3단계_CPA'] = (df_merge_result['비용_최근7일']/df_merge_result['created_최근7일_3단계']).replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
    df_merge_result['최근3일_견적요청_CPA'] = (df_merge_result['비용_최근3일']/df_merge_result['created_최근3일_견적요청']).replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
    df_merge_result['최근7일_견적요청_CPA'] = (df_merge_result['비용_최근7일']/df_merge_result['created_최근7일_견적요청']).replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
    df_merge_result_test = pd.merge(df_merge_result_test,df_merge_result[['publisher.name_facebook_filtering','A_columns','B_columns','비용_최근3일','비용_최근7일','최근3일_CPI','최근7일_CPI','최근3일_3단계_CPA','최근7일_3단계_CPA','최근3일_견적요청_CPA','최근7일_견적요청_CPA']],'left',on=['publisher.name_facebook_filtering','A_columns','B_columns'],suffixes=('','_'+str(i)+'일 전'))

In [ ]:
yesterday = (pd.to_datetime(today) - pd.Timedelta('1 days')).strftime("%Y-%m-%d")

def test(a):
    b = df_merge_facebook[(df_merge_facebook['****date']>=(pd.to_datetime(threedaysago) - pd.Timedelta(a+'day')))&(df_merge_facebook['****date']<=(pd.to_datetime(yesterday)- pd.Timedelta(a+'day')))].pivot_table(index='Adset ID',values=['****date'],aggfunc='count').reset_index()
    bb = df_merge_facebook[(df_merge_facebook['****date']>=(pd.to_datetime(threedaysago) - pd.Timedelta(a+'day')))&(df_merge_facebook['****date']<=(pd.to_datetime(yesterday)- pd.Timedelta(a+'day')))&(df_merge_facebook['견적요청 3단계']<=(pd.to_datetime(yesterday)- pd.Timedelta(str(int(a)-1)+'day')))].pivot_table(index='Adset ID',values=['견적요청 3단계'],aggfunc='count').reset_index()
    bbb = df_merge_facebook[(df_merge_facebook['****date']>=(pd.to_datetime(threedaysago) - pd.Timedelta(a+'day')))&(df_merge_facebook['****date']<=(pd.to_datetime(yesterday)- pd.Timedelta(a+'day')))&(df_merge_facebook['견적요청']<=(pd.to_datetime(yesterday)- pd.Timedelta(str(int(a)-1)+'day')))].pivot_table(index='Adset ID',values=['견적요청'],aggfunc='count').reset_index()
    c = df_merge_facebook[(df_merge_facebook['****date']>=(pd.to_datetime(weekago)- pd.Timedelta(a+'day')))&(df_merge_facebook['****date']<=(pd.to_datetime(yesterday)- pd.Timedelta(a+'day')))].pivot_table(index='Adset ID',values=['****date'],aggfunc='count').reset_index()
    cc = df_merge_facebook[(df_merge_facebook['****date']>=(pd.to_datetime(weekago)- pd.Timedelta(a+'day')))&(df_merge_facebook['****date']<=(pd.to_datetime(yesterday)- pd.Timedelta(a+'day')))&(df_merge_facebook['견적요청 3단계']<=(pd.to_datetime(yesterday)- pd.Timedelta(str(int(a)-1)+'day')))].pivot_table(index='Adset ID',values=['견적요청 3단계'],aggfunc='count').reset_index()
    ccc = df_merge_facebook[(df_merge_facebook['****date']>=(pd.to_datetime(weekago)- pd.Timedelta(a+'day')))&(df_merge_facebook['****date']<=(pd.to_datetime(yesterday)- pd.Timedelta(a+'day')))&(df_merge_facebook['견적요청']<=(pd.to_datetime(yesterday)- pd.Timedelta(str(int(a)-1)+'day')))].pivot_table(index='Adset ID',values=['견적요청'],aggfunc='count').reset_index()

    d = pd.merge(b,bb,'left',on='Adset ID',suffixes=('','_y'))
    d = pd.merge(d,bbb,'left',on='Adset ID',suffixes=('','_y'))
    d = pd.merge(d,c,'left',on='Adset ID',suffixes=('_최근3일_'+a,'_최근7일_'+a))
    d = pd.merge(d,cc,'left',on='Adset ID',suffixes=('_최근3일_'+a,'_최근7일_'+a))
    d = pd.merge(d,ccc,'left',on='Adset ID',suffixes=('_최근3일_'+a,'_최근7일_'+a))
    return d

def spent(a):
    df_index_spent_3days = df_facebook[(df_facebook['보고 시작']>=(pd.to_datetime(threedaysago)-pd.Timedelta(a+'day')))&(df_facebook['보고 시작']<=(pd.to_datetime(yesterday)-pd.Timedelta(a+'day')))].pivot_table(index='Adset ID',values='지출 금액 (KRW)',aggfunc='sum').reset_index()
    df_index_spent_7days = df_facebook[(df_facebook['보고 시작']>=(pd.to_datetime(weekago)-pd.Timedelta(a+'day')))&(df_facebook['보고 시작']<=(pd.to_datetime(yesterday)-pd.Timedelta(a+'day')))].pivot_table(index='Adset ID',values='지출 금액 (KRW)',aggfunc='sum').reset_index()
    df_index_spent_14days = df_facebook[(df_facebook['보고 시작']>=(pd.to_datetime(twoweeksago)-pd.Timedelta(a+'day')))&(df_facebook['보고 시작']<=(pd.to_datetime(yesterday)-pd.Timedelta(a+'day')))].pivot_table(index='Adset ID',values='지출 금액 (KRW)',aggfunc='sum').reset_index()
    df_index_merge = pd.merge(df_index_spent_3days,df_index_spent_7days,'left',on='Adset ID',suffixes=('','_최근7일_'+a))
    df_index_merge = pd.merge(df_index_merge,df_index_spent_14days,'left',on='Adset ID',suffixes=('_최근3일_'+a,'_최근14일_'+a))
    return df_index_merge

df_index = df_facebook[(df_facebook['보고 시작']>=weekago)&(df_facebook['노출']>=1)].sort_values(['보고 시작'],ascending=False).pivot_table(index=['Adset ID','캠페인 이름','광고 세트 이름'],values=['예산','만든 날짜','게재'],aggfunc='first').reset_index()

for i in range(7):
    df_index = pd.merge(df_index,test(str(6-i)),'left',on='Adset ID',suffixes=('','_y')).fillna(0)
    df_index = pd.merge(df_index,spent(str(6-i)),'left',on='Adset ID',suffixes=('','_y')).fillna(0)
    
for i in range(7):
    df_index['최근3일_CPI_'+str(6-i)] =  np.where(df_index['****date_최근3일_'+str(6-i)]==0,0,df_index['지출 금액 (KRW)_최근3일_'+str(6-i)]/df_index['****date_최근3일_'+str(6-i)])
for i in range(7):
    df_index['최근7일_CPI_'+str(6-i)] =  np.where(df_index['****date_최근7일_'+str(6-i)]==0,0,df_index['지출 금액 (KRW)_최근7일_'+str(6-i)]/df_index['****date_최근7일_'+str(6-i)])
for i in range(7):
    df_index['최근3일_3단계_CPA_'+str(6-i)] =  np.where(df_index['견적요청 3단계_최근3일_'+str(6-i)]==0,0,df_index['지출 금액 (KRW)_최근3일_'+str(6-i)]/df_index['견적요청 3단계_최근3일_'+str(6-i)])
for i in range(7):
    df_index['최근7일_3단계_CPA_'+str(6-i)] =  np.where(df_index['견적요청 3단계_최근7일_'+str(6-i)]==0,0,df_index['지출 금액 (KRW)_최근7일_'+str(6-i)]/df_index['견적요청 3단계_최근7일_'+str(6-i)])
for i in range(7):
    df_index['최근3일_견적요청_CPA_'+str(6-i)] =  np.where(df_index['견적요청_최근3일_'+str(6-i)]==0,999999,df_index['지출 금액 (KRW)_최근3일_'+str(6-i)]/df_index['견적요청_최근3일_'+str(6-i)])
for i in range(7):
    df_index['최근7일_견적요청_CPA_'+str(6-i)] =  np.where(df_index['견적요청_최근7일_'+str(6-i)]==0,999999,df_index['지출 금액 (KRW)_최근7일_'+str(6-i)]/df_index['견적요청_최근7일_'+str(6-i)])

In [ ]:
#### 광고 그룹 scoring

for i in range(7):
    i = "_"+str(i)
    df_index['overCPA_최근7일'+i] = np.where(df_index['최근7일_견적요청_CPA'+i]>100000,'Bad',
                                       np.where(df_index['최근7일_견적요청_CPA'+i]==0,'NotYet','Good'))
    df_index['overCPA_최근3일'+i] = np.where(df_index['최근3일_견적요청_CPA'+i]>100000,'Bad',
                                       np.where(df_index['최근3일_견적요청_CPA'+i]==0,'NotYet','Good'))

    df_index['CPI_트렌드'+i] = np.where(((df_index['최근7일_CPI'+i]==0)|(df_index['최근3일_CPI'+i]==0)),0,
                                  np.where((df_index['최근7일_CPI'+i]>df_index['최근3일_CPI'+i]),1,0))
    df_index['3단계_CPA_트렌드'+i] = np.where(((df_index['최근7일_3단계_CPA'+i]==0)|(df_index['최근3일_3단계_CPA'+i]==0)),0,
                                  np.where(df_index['최근7일_3단계_CPA'+i]>df_index['최근3일_3단계_CPA'+i],1,0))
    df_index['견적요청_CPA_트렌드'+i] = np.where(((df_index['최근7일_견적요청_CPA'+i]==0)|(df_index['최근3일_견적요청_CPA'+i]==0)),0,
                                        np.where(df_index['최근7일_견적요청_CPA'+i]>df_index['최근3일_견적요청_CPA'+i],1,0))

    df_index['최근7일_CPI_성과'+i] = np.where(df_index['최근7일_CPI'+i]==0,0,
                                      np.where(df_index['최근7일_CPI'+i]>df_index[(df_index['overCPA_최근7일'+i]=='Good')][['최근7일_CPI'+i,'최근7일_3단계_CPA'+i,'최근7일_견적요청_CPA'+i]].mean()[0],0,1))
    df_index['최근7일_3단계_CPA_성과'+i] = np.where(df_index['최근7일_3단계_CPA'+i]==0,0,
                                          np.where(df_index['최근7일_3단계_CPA'+i]>df_index[(df_index['overCPA_최근7일'+i]=='Good')][['최근7일_CPI'+i,'최근7일_3단계_CPA'+i,'최근7일_견적요청_CPA'+i]].mean()[1],0,2))
    df_index['최근7일_견적요청_CPA_성과'+i] = np.where(df_index['최근7일_견적요청_CPA'+i]==0,0,
                                          np.where(df_index['최근7일_견적요청_CPA'+i]>df_index[(df_index['overCPA_최근7일'+i]=='Good')][['최근7일_CPI'+i,'최근7일_3단계_CPA'+i,'최근7일_견적요청_CPA'+i]].mean()[2],0,3))
    df_index['최근3일_CPI_성과'+i] = np.where(df_index['최근3일_CPI'+i]==0,0,
                                      np.where(df_index['최근3일_CPI'+i]>df_index[(df_index['overCPA_최근3일'+i]=='Good')][['최근3일_CPI'+i,'최근3일_3단계_CPA'+i,'최근3일_견적요청_CPA'+i]].mean()[0],0,1))
    df_index['최근3일_3단계_CPA_성과'+i] = np.where(df_index['최근3일_3단계_CPA'+i]==0,0,
                                          np.where(df_index['최근3일_3단계_CPA'+i]>df_index[(df_index['overCPA_최근3일'+i]=='Good')][['최근3일_CPI'+i,'최근3일_3단계_CPA'+i,'최근3일_견적요청_CPA'+i]].mean()[1],0,2))
    df_index['최근3일_견적요청_CPA_성과'+i] = np.where(df_index['최근3일_견적요청_CPA'+i]==0,0,
                                          np.where(df_index['최근3일_견적요청_CPA'+i]>df_index[(df_index['overCPA_최근3일'+i]=='Good')][['최근3일_CPI'+i,'최근3일_3단계_CPA'+i,'최근3일_견적요청_CPA'+i]].mean()[2],0,3))

    df_index['성과점수합'+i] = df_index.loc[:,'최근7일_CPI_성과'+i:'최근3일_견적요청_CPA_성과'+i].sum(axis=1)
    df_index['트렌드점수합'+i] = df_index.loc[:,'CPI_트렌드'+i:'견적요청_CPA_트렌드'+i].sum(axis=1)
    
    df_index['CPA>150000'+i] = np.where(((df_index['최근3일_견적요청_CPA'+i]>150000)&(df_index['최근7일_견적요청_CPA'+i]>150000)),2,
                                     np.where(((df_index['최근3일_견적요청_CPA'+i]>150000)|(df_index['최근7일_견적요청_CPA'+i]>150000)),1,0))
    
    df_index['예산조정치'+i] = np.where(((df_index['성과점수합'+i]==0)&(df_index['최근7일_견적요청_CPA'+i]>120000)&(df_index['최근3일_견적요청_CPA'+i]>120000)),-1,
                                np.where(df_index['성과점수합'+i]<=3,0,df_index['성과점수합'+i]*10000 + df_index['트렌드점수합'+i]*10000))
    df_index['예산조정치'+i] = np.where(((df_index['예산조정치'+i]==-1)&(df_index['지출 금액 (KRW)_최근7일'+i]<100000)),0,df_index['예산조정치'+i])
    ####홀딩세트 조정
    df_index['예산조정치'+i] = np.where(((df_index['예산조정치'+i]==0)&(df_index['overCPA_최근3일'+i]=='Good')),50000,df_index['예산조정치'+i])
    df_index['예산조정치'+i] = np.where(((df_index['예산조정치'+i]==0)&(df_index['지출 금액 (KRW)_최근14일'+i]>=100000)),-1,df_index['예산조정치'+i])
    ####off세트 조정
    df_index['예산조정치'+i] = np.where(((df_index['예산조정치'+i]==-1)&(df_index['overCPA_최근7일'+i]=='Good')),30000,df_index['예산조정치'+i])
    ####up세트 조정
    df_index['예산조정치'+i] = np.where(((df_index['예산조정치'+i]>50000)&((df_index['overCPA_최근3일'+i]=='Bad')|(df_index['overCPA_최근7일'+i]=='Bad'))),0,df_index['예산조정치'+i])
    df_index['예산조정'+i] = np.where((df_index['최근3일_견적요청_CPA'+i]>200000)|(df_index['CPA>150000'+i]==2),0.5,
                                 np.where((df_index['CPA>150000'+i]==1),0.7,
                                          np.where((df_index['성과점수합'+i]==0),5000,
                                                   np.where(((df_index['성과점수합'+i]>0)&(df_index['성과점수합'+i]<4)),10000,20000))))

    df_index['예산'] = df_index['예산'].astype(int)

#### 성과별 예산 조정
df_index['ResultBudget'] = np.where(df_index['예산조정_0']<=1,((df_index['예산'] * df_index['예산조정_0'])/100).astype(int)*100,
                                   df_index['예산']+((df_index['성과점수합_0'])+(df_index['트렌드점수합_0']))*(df_index['예산조정_0']))
#### 세트 예산 한도
df_index['ResultBudget'] = np.where(df_index['ResultBudget']>=700000,700000,df_index['ResultBudget'])

#### 예산 감액 조정 오류 방지
df_index['ResultBudget'] = np.where(((df_index['지출 금액 (KRW)_최근3일_0']<=40000)&(df_index['예산조정_0']>1)),df_index['예산'],df_index['ResultBudget'])


df_index['광고 세트 현황'] = np.where((pd.to_datetime(yesterday)-pd.to_datetime(df_index['만든 날짜'])).dt.days==2,'테스트_3일차',
                           np.where((pd.to_datetime(yesterday)-pd.to_datetime(df_index['만든 날짜'])).dt.days==6,'테스트_종료',
                                   np.where((pd.to_datetime(yesterday)-pd.to_datetime(df_index['만든 날짜'])).dt.days<6,'테스트_중','최적화_단계')))
df_index = df_index[(df_index['광고 세트 현황']=='최적화_단계')|(df_index['광고 세트 이름'].str.contains('Copy',regex=True)==True)]
df_index['Status'] = 'ACTIVE'
df_index['Ads'] = np.where(df_index['캠페인 이름'].str[:5]=='****',df_index['광고 세트 이름'].str[-6:] ,df_index['광고 세트 이름'].str[4:12] )

In [ ]:
df_index = df_index[(df_index['지출 금액 (KRW)_최근7일_0']>0)]
df_index = df_index[(df_index['게재']=='active')&(df_index['예산조정_0']!=1)].reset_index()
df_index['ResultBudget'] = np.where(df_index['ResultBudget']<=1111,1111,df_index['ResultBudget'])
df_index['Status'] = np.where(df_index['ResultBudget']==1111,'PAUSED','ACTIVE')

#  성과 분석 결과를 토대로 FBIG 예산 조정 및 on/off 자동화

In [ ]:
#### 페이스북 조정 1차
#### 한 번에 돌리면 중단되는 경우 많아서 1/2차로 나눔
for i in range(df_index.iloc[:30,].shape[0]):

    Do = AdSet(str(df_index['Adset ID'][i])).remote_update(
        params={
        'status' : df_index['Status'][i],
        'daily_budget' : str(df_index['ResultBudget'][i])
    }
    )
    sleep(30)
    print(Do)

In [ ]:
#### 페이스북 조정 2차
for i in range(df_index.iloc[30:,].shape[0]-1):

    Do = AdSet(str(df_index['Adset ID'][i+31])).remote_update(
        params={
        'status' : df_index['Status'][i+31],
        'daily_budget' : str(df_index['ResultBudget'][i+31])
    }
    )
    sleep(30)
    print(Do)

# 데일리 팀 내 결과 공유

In [ ]:
#### 데일리 팀 내 결과 공유

df_index[(df_index['게재']=='active')][['Adset ID','캠페인 이름','광고 세트 이름','Ads',"지출 금액 (KRW)_최근3일_0","지출 금액 (KRW)_최근7일_0",'최근3일_견적요청_CPA_0','최근7일_견적요청_CPA_0',"3단계_CPA_트렌드_0", "견적요청_CPA_트렌드_0", "최근7일_CPI_성과_0", "최근7일_3단계_CPA_성과_0", "최근7일_견적요청_CPA_성과_0", "최근3일_CPI_성과_0", "최근3일_3단계_CPA_성과_0", "최근3일_견적요청_CPA_성과_0",'예산조정치_0','예산조정_0','예산','ResultBudget']].sort_values(['ResultBudget'],ascending=False).to_excel('/path/%s_페이스북_예산조정결과_2.xlsx' %today)
df_merge_result_test[df_merge_result_test_columns].to_excel('/path/%s_광고채널_CPA.xlsx' %today)
df_result_raw_data.to_excel('/path/%s_로데이터.xlsx' %today)

channel = '****'
token = '****'
with open('/path/%s_로데이터.xlsx' %today, 'rb') as file:
    url = 'https://slack.com/api/files.upload'
    data = {
        'token': token,
        'channels': channel,
        'filename': file.name,
    }
    files = {
        'file': ('dummy', file),
    }
    response = requests.post(url, data=data, files=files)
    print(response.text)
    
with open('/path/%s_광고채널_CPA.xlsx' %today, 'rb') as file:
    url = 'https://slack.com/api/files.upload'
    data = {
        'token': token,
        'channels': channel,
        'filename': file.name,
    }
    files = {
        'file': ('dummy', file),
    }
    response = requests.post(url, data=data, files=files)
    print(response.text)

with open('/path/%s_페이스북_예산조정결과.xlsx' %today, 'rb') as file:
    url = 'https://slack.com/api/files.upload'
    data = {
        'token': token,
        'channels': channel,
        'filename': file.name,
    }
    files = {
        'file': ('dummy', file),
    }
    response = requests.post(url, data=data, files=files)
    print(response.text)